In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from efficientnet_pytorch import EfficientNet

In [ ]:
# параметры
data_dir = "/content/drive/MyDrive/Dataset_all_class"
batch_size = 32
image_size = (224, 224)
learning_rate = 0.001
epochs = 20
validation_split = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(data_dir, transform=transform)


num_val = int(len(dataset) * validation_split)  #разделение датасета
num_train = len(dataset) - num_val
train_dataset, val_dataset = random_split(dataset, [num_train, num_val])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

class_names = dataset.classes #получение классов
num_classes = len(class_names)


model = EfficientNet.from_pretrained('efficientnet-b3')  #создание модели
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

best_val_accuracy = 0.0
best_model_path = None

for epoch in range(epochs):   #тренировка модели
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}", unit="batch") as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.set_postfix({"loss": train_loss / (pbar.n + 1), "accuracy": correct / total})
            pbar.update(1)

    train_loss = train_loss / len(train_loader)
    train_accuracy = correct / total
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    # Оценка модели на валидационном наборе
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    y_true_val = []
    y_pred_val = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            y_true_val.extend(labels.cpu().numpy())
            y_pred_val.extend(predicted.cpu().numpy())

    val_loss = val_loss / len(val_loader)
    val_accuracy = correct_val / total_val

    print(f"Epoch: {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy: # Сохранение лучшей модели(без early stopping)
        best_val_accuracy = val_accuracy
        best_model_path = os.path.join(save_dir, "best_model.pth")
        torch.save(model.state_dict(), best_model_path)

print("Training complete. Best model saved.")




# Загрузка лучшей модели и оценка на валидационном наборе
if best_model_path:
    model.load_state_dict(torch.load(best_model_path))
model.eval()
correct = 0
total = 0
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.4f}")
print(classification_report(y_true, y_pred, target_names=class_names))
print(confusion_matrix(y_true, y_pred))

Loaded pretrained weights for efficientnet-b3


Epoch 1: 100%|██████████| 257/257 [13:59<00:00,  3.26s/batch, loss=0.939, accuracy=0.719]

Epoch: 1, Train Loss: 0.9391, Train Accuracy: 0.7194


Epoch: 1, Val Loss: 0.7456, Val Accuracy: 0.7646


Epoch 2: 100%|██████████| 257/257 [04:30<00:00,  1.05s/batch, loss=0.495, accuracy=0.846]

...

Epoch 20: 100%|██████████| 257/257 [04:11<00:00,  1.02batch/s, loss=0.0768, accuracy=0.977]

Epoch: 20, Train Loss: 0.0768, Train Accuracy: 0.9766


Epoch: 20, Val Loss: 0.8580, Val Accuracy: 0.8271
Training complete. Best model saved.


<ipython-input-29-3a6b6a247639>:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Validation Accuracy: 0.8623
                           precision    recall  f1-score   support

               Археология       0.85      0.97      0.91        96
                  Графика       0.98      0.80      0.88       110
                      ДПИ       0.58      0.64      0.61       132
                Документы       0.91      0.97      0.94       103
Естественнонауч.коллекция       0.91      0.94      0.93       225
                 Живопись       0.67      0.89      0.76        97
              Минералогия       0.97      0.97      0.97       113
              Нумизматика       0.91      0.94      0.92       154
                   Оружие       1.00      0.83      0.90        86
       Печатная продукция       0.92      0.80      0.85       129
                   Прочие       0.84      0.73      0.78       206
             Редкие книги       0.96      0.96      0.96       257
               Скульптура       0.87      0.74      0.80       126
                  Техника       0